# Inspect `test_bust_full_gens_summary_and_overall_pass` Results

This notebook runs the test function and displays all returned dataframes and variables in an easy-to-read format.


In [17]:
# Import necessary modules
from typing import Callable, Dict, List, Optional, Sequence, Tuple
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent.parent.parent
sys.path.insert(0, str(project_root))

import molgen3D.evaluation.posebusters_check as posebusters_check_module
from molgen3D.evaluation.posebusters_check import bust_full_gens

# Import test utilities
POSEBUSTERS_BOOL_COLUMNS: Tuple[str, ...] = (
    "mol_pred_loaded",
    "sanitization",
    "inchi_convertible",
    "all_atoms_connected",
)
BoolMatrix = Sequence[Sequence[bool]]
ErrorMap = Dict[int, str]
PatchReportFunc = Callable[[BoolMatrix, Optional[ErrorMap]], pd.DataFrame]

print("✓ Imports successful")


✓ Imports successful


In [18]:
# Helper function to build conformer payload with dummy RDKit molecules
def _build_conformer_payload(
    smiles_counts: Sequence[Tuple[str, int]]
) -> Dict[str, List]:
    """Return placeholder conformer lists for each SMILES with dummy RDKit molecules."""
    from rdkit import Chem
    from rdkit.Chem import AllChem
    
    payload: Dict[str, List] = {}
    for smiles, count in smiles_counts:
        # Create dummy molecules for each conformer
        mols = []
        for _ in range(count):
            # Create a simple molecule from SMILES
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                # If SMILES parsing fails, create a dummy molecule
                mol = Chem.MolFromSmiles("C")
            # Add hydrogens and generate 3D coordinates
            mol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol, randomSeed=42)
            mols.append(mol)
        payload[smiles] = mols
    return payload

# Global variable to store patch handle
_patch_handle = None

# Global variable to store per_conformer_df from bust_full_gens
_captured_per_conformer_df = None

# Helper function to patch posebusters report (from test file)
def setup_patch_posebusters_report(
    bool_matrix: BoolMatrix,
    error_map: Optional[ErrorMap] = None,
) -> None:
    """Patch ``_collect_posebusters_report`` with a deterministic frame."""
    global _patch_handle
    
    if not bool_matrix:
        raise ValueError("bool_matrix must contain at least one conformer.")
    error_map = error_map or {}
    rows: List[Dict[str, object]] = []
    for idx, bools in enumerate(bool_matrix):
        if len(bools) != len(POSEBUSTERS_BOOL_COLUMNS):
            raise ValueError(
                "Each bool vector must match POSEBUSTERS_BOOL_COLUMNS."
            )
        row: Dict[str, object] = {
            "conformer_index": idx,
            "error": error_map.get(idx, ""),
        }
        for col, value in zip(POSEBUSTERS_BOOL_COLUMNS, bools):
            row[col] = bool(value)
        rows.append(row)
    frame = pd.DataFrame(rows)
    for col in POSEBUSTERS_BOOL_COLUMNS:
        frame[col] = frame[col].astype(bool)

    def _fake_report(
        rd_confs: Sequence[object],
        num_workers: int,
        full_report: bool,
        config: str,
    ) -> pd.DataFrame:
        assert len(rd_confs) == len(bool_matrix)
        return frame.copy()

    # Use unittest.mock instead of pytest monkeypatch for notebook
    import unittest.mock
    if _patch_handle is not None:
        _patch_handle.stop()
    _patch_handle = unittest.mock.patch.object(
        posebusters_check_module,
        "_collect_posebusters_report",
        _fake_report,
    )
    _patch_handle.start()

# Wrapper function to capture per_conformer_df from bust_full_gens
def bust_full_gens_with_capture(
    smiles_to_confs: Dict[str, Sequence],
    *,
    num_workers: int = 8,
    config: str = "mol",
    full_report: bool = False,
    fail_threshold: float = 0.0,
):
    """
    Wrapper around bust_full_gens that captures the intermediate per_conformer_df.
    
    This function monkey-patches the internal _collect_posebusters_report to capture
    per_conformer_df before it gets aggregated into per_smiles_df.
    """
    global _captured_per_conformer_df
    from molgen3D.evaluation.posebusters_check import (
        _collect_posebusters_report,
        _compute_pass_fraction,
    )
    import unittest.mock
    
    # Store original function
    original_collect = _collect_posebusters_report
    
    # Track if we've captured it
    captured_df = None
    
    def capturing_collect(*args, **kwargs):
        """Wrapper that captures the per_conformer_df."""
        nonlocal captured_df
        df = original_collect(*args, **kwargs)
        captured_df = df.copy() if df is not None else None
        return df
    
    # Patch the function temporarily
    with unittest.mock.patch.object(
        posebusters_check_module,
        "_collect_posebusters_report",
        capturing_collect,
    ):
        # Call the original function
        result = bust_full_gens(
            smiles_to_confs,
            num_workers=num_workers,
            config=config,
            full_report=full_report,
            fail_threshold=fail_threshold,
        )
    
    # Now we need to reconstruct per_conformer_df with the SMILES column
    # by calling the same logic that bust_full_gens uses
    if captured_df is not None and not captured_df.empty:
        # Reconstruct the flattened list and mapping (same logic as bust_full_gens)
        flattened: List = []
        flattened_idx_to_smiles_idx: List[int] = []
        unique_smiles: List[str] = []
        smiles_to_idx: Dict[str, int] = {}
        
        for smiles, confs in smiles_to_confs.items():
            if not confs:
                continue
            valid_confs = [mol for mol in confs if mol is not None]
            if not valid_confs:
                continue
            
            smiles_idx = smiles_to_idx.setdefault(smiles, len(unique_smiles))
            if smiles_idx == len(unique_smiles):
                unique_smiles.append(smiles)
            
            flattened.extend(valid_confs)
            flattened_idx_to_smiles_idx.extend([smiles_idx] * len(valid_confs))
        
        # Add the columns that bust_full_gens adds
        per_conformer_df = captured_df.copy()
        pass_fraction = _compute_pass_fraction(per_conformer_df)
        per_conformer_df["pass_fraction"] = pass_fraction
        per_conformer_df["pass_bool"] = (np.isclose(pass_fraction, 1.0)).astype(float)
        per_conformer_df["smiles"] = per_conformer_df["conformer_index"].map(
            lambda idx: unique_smiles[flattened_idx_to_smiles_idx[int(idx)]]
        )
        
        _captured_per_conformer_df = per_conformer_df
    else:
        _captured_per_conformer_df = None
    
    return result

print("✓ Helper functions defined")


✓ Helper functions defined


In [19]:
# Set up test data (exactly as in the test function)
smiles_counts: List[Tuple[str, int]] = [
    ("NC(=O)C1CCC1", 2),
    ("CC(C)(C)n1cn[nH]c1=S", 3),
    ("Cc1c(C#N)c2ccccc2n1C", 1),
]
payload = _build_conformer_payload(smiles_counts)

# Define the boolean matrix (pass/fail for each conformer)
bool_matrix: BoolMatrix = [
    # NC(=O)C1CCC1 conformers
    (True, True, True, True),
    (True, True, True, True),
    # CC(C)(C)n1cn[nH]c1=S conformers
    (True, True, True, True),
    (True, False, True, False),
    (False, False, False, False),  # error row
    # Cc1c(C#N)c2ccccc2n1C conformer
    (True, False, False, False),
]

# Set up the patch
setup_patch_posebusters_report(bool_matrix, error_map={4: "posebusters_error"})

print("✓ Test data and patch configured")
print(f"  - SMILES counts: {smiles_counts}")
print(f"  - Total conformers: {sum(count for _, count in smiles_counts)}")
print(f"  - Boolean matrix shape: {len(bool_matrix)} conformers × {len(bool_matrix[0])} checks")


✓ Test data and patch configured
  - SMILES counts: [('NC(=O)C1CCC1', 2), ('CC(C)(C)n1cn[nH]c1=S', 3), ('Cc1c(C#N)c2ccccc2n1C', 1)]
  - Total conformers: 6
  - Boolean matrix shape: 6 conformers × 4 checks


In [20]:
# Run bust_full_gens with single worker (using wrapper to capture per_conformer_df)
print("Running bust_full_gens with num_workers=1...")
(
    per_smiles_single,
    summary_single,
    overall_single,
    fail_single,
    err_single,
) = bust_full_gens_with_capture(
    payload,
    num_workers=1,
    config="mol",
    full_report=False,
    fail_threshold=0.2,
)

# Store the captured per_conformer_df
per_conformer_single = _captured_per_conformer_df

print("✓ Single worker run completed")
print(f"  - Overall pass rate: {overall_single:.6f} ({overall_single*100:.2f}%)")
print(f"  - Failed SMILES: {fail_single}")
print(f"  - Error SMILES: {err_single}")


Running bust_full_gens with num_workers=1...
✓ Single worker run completed
  - Overall pass rate: 0.500000 (50.00%)
  - Failed SMILES: ['CC(C)(C)n1cn[nH]c1=S', 'Cc1c(C#N)c2ccccc2n1C']
  - Error SMILES: ['CC(C)(C)n1cn[nH]c1=S']


In [21]:
# Run bust_full_gens with multiple workers (using wrapper to capture per_conformer_df)
print("Running bust_full_gens with num_workers=3...")
(
    per_smiles_multi,
    summary_multi,
    overall_multi,
    fail_multi,
    err_multi,
) = bust_full_gens_with_capture(
    payload,
    num_workers=3,
    config="mol",
    full_report=False,
    fail_threshold=0.2,
)

# Store the captured per_conformer_df
per_conformer_multi = _captured_per_conformer_df

print("✓ Multi worker run completed")
print(f"  - Overall pass rate: {overall_multi:.6f} ({overall_multi*100:.2f}%)")
print(f"  - Failed SMILES: {fail_multi}")
print(f"  - Error SMILES: {err_multi}")

# Verify they match
print("\n✓ Verification:")
print(f"  - Per-SMILES dataframes match: {per_smiles_single.equals(per_smiles_multi)}")
print(f"  - Summary dataframes match: {summary_single.equals(summary_multi)}")
print(f"  - Overall pass rates match: {np.isclose(overall_single, overall_multi, rtol=1e-6)}")
print(f"  - Fail lists match: {fail_single == fail_multi}")
print(f"  - Error lists match: {err_single == err_multi}")


Running bust_full_gens with num_workers=3...
✓ Multi worker run completed
  - Overall pass rate: 0.500000 (50.00%)
  - Failed SMILES: ['CC(C)(C)n1cn[nH]c1=S', 'Cc1c(C#N)c2ccccc2n1C']
  - Error SMILES: ['CC(C)(C)n1cn[nH]c1=S']

✓ Verification:
  - Per-SMILES dataframes match: True
  - Summary dataframes match: True
  - Overall pass rates match: True
  - Fail lists match: True
  - Error lists match: True


## Per-Conformer DataFrame

This dataframe contains one row per conformer with detailed statistics.


In [22]:
# Display per-conformer dataframe with nice formatting
print("=" * 80)
print("PER-CONFORMER DATAFRAME (Single Worker)")
print("=" * 80)
if per_conformer_single is not None:
    print(f"\nShape: {per_conformer_single.shape}")
    print(f"\nColumns: {list(per_conformer_single.columns)}")
    print("\n" + "=" * 80)
    display(per_conformer_single)
    
    # Show data types
    print("\nData types:")
    print(per_conformer_single.dtypes)
    
    # Show summary statistics
    print("\n" + "=" * 80)
    print("SUMMARY STATISTICS")
    print("=" * 80)
    print(f"Total conformers: {len(per_conformer_single)}")
    if "pass_bool" in per_conformer_single.columns:
        print(f"Conformers passing all checks: {per_conformer_single['pass_bool'].sum():.0f} ({per_conformer_single['pass_bool'].mean()*100:.2f}%)")
    if "smiles" in per_conformer_single.columns:
        print(f"\nConformers per SMILES:")
        smiles_counts = per_conformer_single["smiles"].value_counts().sort_index()
        for smiles, count in smiles_counts.items():
            print(f"  {smiles}: {count}")
else:
    print("No per_conformer_df captured (empty result)")

PER-CONFORMER DATAFRAME (Single Worker)

Shape: (6, 9)

Columns: ['conformer_index', 'error', 'mol_pred_loaded', 'sanitization', 'inchi_convertible', 'all_atoms_connected', 'pass_fraction', 'pass_bool', 'smiles']



,conformer_index,error,mol_pred_loaded,sanitization,inchi_convertible,all_atoms_connected,pass_fraction,pass_bool,smiles
0,0,,True,True,True,True,1.00,1.0,NC(=O)C1CCC1
1,1,,True,True,True,True,1.00,1.0,NC(=O)C1CCC1
2,2,,True,True,True,True,1.00,1.0,CC(C)(C)n1cn[nH]c1=S
3,3,,True,False,True,False,0.50,0.0,CC(C)(C)n1cn[nH]c1=S
4,4,posebusters_error,False,False,False,False,0.00,0.0,CC(C)(C)n1cn[nH]c1=S
5,5,,True,False,False,False,0.25,0.0,Cc1c(C#N)c2ccccc2n1C



Data types:
conformer_index          int64
error                   object
mol_pred_loaded           bool
sanitization              bool
inchi_convertible         bool
all_atoms_connected       bool
pass_fraction          float64
pass_bool              float64
smiles                  object
dtype: object

SUMMARY STATISTICS
Total conformers: 6
Conformers passing all checks: 3 (50.00%)

Conformers per SMILES:
  CC(C)(C)n1cn[nH]c1=S: 3
  Cc1c(C#N)c2ccccc2n1C: 1
  NC(=O)C1CCC1: 2


## Per-Conformer DataFrame Comparison

Compare single vs multi-worker results for per-conformer dataframe.


In [23]:
# Compare per_conformer dataframes from single vs multi-worker runs
print("=" * 80)
print("PER-CONFORMER DATAFRAME COMPARISON")
print("=" * 80)

if per_conformer_single is not None and per_conformer_multi is not None:
    # Compare shapes and columns
    print(f"\nSingle worker shape: {per_conformer_single.shape}")
    print(f"Multi worker shape: {per_conformer_multi.shape}")
    print(f"Shapes match: {per_conformer_single.shape == per_conformer_multi.shape}")
    
    # Compare dataframes (excluding conformer_index which might differ)
    cols_to_compare = [col for col in per_conformer_single.columns 
                      if col != "conformer_index"]
    single_subset = per_conformer_single[cols_to_compare].sort_values("smiles").reset_index(drop=True)
    multi_subset = per_conformer_multi[cols_to_compare].sort_values("smiles").reset_index(drop=True)
    
    print(f"\nDataframes match (excluding conformer_index): {single_subset.equals(multi_subset)}")
    
    if not single_subset.equals(multi_subset):
        print("\nDifferences found:")
        diff_mask = ~(single_subset == multi_subset).all(axis=1)
        if diff_mask.any():
            print(single_subset[diff_mask].compare(multi_subset[diff_mask]))
else:
    print("One or both per_conformer dataframes are None")


PER-CONFORMER DATAFRAME COMPARISON

Single worker shape: (6, 9)
Multi worker shape: (6, 9)
Shapes match: True

Dataframes match (excluding conformer_index): True


## Per-SMILES DataFrame

This dataframe contains one row per SMILES string with aggregated statistics.


In [24]:
# Display per-SMILES dataframe with nice formatting
print("=" * 80)
print("PER-SMILES DATAFRAME")
print("=" * 80)
print(f"\nShape: {per_smiles_single.shape}")
print(f"\nColumns: {list(per_smiles_single.columns)}")
print("\n" + "=" * 80)
display(per_smiles_single)

# Show data types
print("\nData types:")
print(per_smiles_single.dtypes)


PER-SMILES DATAFRAME

Shape: (3, 8)

Columns: ['mol_pred_loaded', 'sanitization', 'inchi_convertible', 'all_atoms_connected', 'pass_percentage', 'smiles', 'num_of_conformers', 'error']



,mol_pred_loaded,sanitization,inchi_convertible,all_atoms_connected,pass_percentage,smiles,num_of_conformers,error
0,0.666667,0.333333,0.666667,0.333333,33.333333,CC(C)(C)n1cn[nH]c1=S,3,posebusters_error
1,1.000000,0.000000,0.000000,0.000000,0.000000,Cc1c(C#N)c2ccccc2n1C,1,
2,1.000000,1.000000,1.000000,1.000000,100.000000,NC(=O)C1CCC1,2,



Data types:
mol_pred_loaded        float64
sanitization           float64
inchi_convertible      float64
all_atoms_connected    float64
pass_percentage        float64
smiles                  object
num_of_conformers        int64
error                   object
dtype: object


## Summary DataFrame

This dataframe contains dataset-wide aggregated statistics (single row).


In [25]:
# Display summary dataframe
print("=" * 80)
print("SUMMARY DATAFRAME")
print("=" * 80)
print(f"\nShape: {summary_single.shape}")
print(f"\nColumns: {list(summary_single.columns)}")
print("\n" + "=" * 80)
display(summary_single)

# Show data types
print("\nData types:")
print(summary_single.dtypes)


SUMMARY DATAFRAME

Shape: (1, 8)

Columns: ['smiles', 'num_smiles', 'num_conformers', 'mol_pred_loaded', 'sanitization', 'inchi_convertible', 'all_atoms_connected', 'pass_percentage']



,smiles,num_smiles,num_conformers,mol_pred_loaded,sanitization,inchi_convertible,all_atoms_connected,pass_percentage
0,ALL,3.0,6.0,0.833333,0.5,0.666667,0.5,50.0



Data types:
smiles                  object
num_smiles             float64
num_conformers         float64
mol_pred_loaded        float64
sanitization           float64
inchi_convertible      float64
all_atoms_connected    float64
pass_percentage        float64
dtype: object


## Scalar Values and Lists


In [26]:
# Display scalar values and lists
print("=" * 80)
print("SCALAR VALUES AND LISTS")
print("=" * 80)

print(f"\n📊 Overall Pass Rate (single worker):")
print(f"   Value: {overall_single}")
print(f"   Percentage: {overall_single * 100:.4f}%")

print(f"\n❌ Failed SMILES (pass_percentage < 80% with fail_threshold=0.2):")
print(f"   Count: {len(fail_single)}")
print(f"   List: {fail_single}")

print(f"\n⚠️  Error SMILES (had PoseBusters runtime errors):")
print(f"   Count: {len(err_single)}")
print(f"   List: {err_single}")

# Show which SMILES had errors in the per_smiles dataframe
if len(err_single) > 0:
    print("\n📋 Error details from per_smiles dataframe:")
    error_rows = per_smiles_single[per_smiles_single['smiles'].isin(err_single)]
    display(error_rows[['smiles', 'error', 'pass_percentage']])


SCALAR VALUES AND LISTS

📊 Overall Pass Rate (single worker):
   Value: 0.5
   Percentage: 50.0000%

❌ Failed SMILES (pass_percentage < 80% with fail_threshold=0.2):
   Count: 2
   List: ['CC(C)(C)n1cn[nH]c1=S', 'Cc1c(C#N)c2ccccc2n1C']

⚠️  Error SMILES (had PoseBusters runtime errors):
   Count: 1
   List: ['CC(C)(C)n1cn[nH]c1=S']

📋 Error details from per_smiles dataframe:


,smiles,error,pass_percentage
0,CC(C)(C)n1cn[nH]c1=S,posebusters_error,33.333333


## Export to CSV Files

Save the dataframes to CSV files for external inspection.


In [27]:
# Save dataframes to CSV files
output_dir = Path("test_posebusters_output")
output_dir.mkdir(exist_ok=True)

# Save per-conformer dataframe
if per_conformer_single is not None:
    per_conformer_path = output_dir / "per_conformer_df.csv"
    per_conformer_single.to_csv(per_conformer_path, index=False)
    print(f"✓ Saved per-conformer dataframe to: {per_conformer_path}")

# Save per-SMILES dataframe
per_smiles_path = output_dir / "per_smiles_df.csv"
per_smiles_single.to_csv(per_smiles_path, index=False)
print(f"✓ Saved per-SMILES dataframe to: {per_smiles_path}")

# Save summary dataframe
summary_path = output_dir / "summary_df.csv"
summary_single.to_csv(summary_path, index=False)
print(f"✓ Saved summary dataframe to: {summary_path}")

# Save scalar values and lists to a text file
results_path = output_dir / "scalar_results.txt"
with open(results_path, 'w') as f:
    f.write("=" * 80 + "\n")
    f.write("SCALAR VALUES AND LISTS\n")
    f.write("=" * 80 + "\n\n")
    f.write(f"Overall Pass Rate: {overall_single} ({overall_single*100:.4f}%)\n\n")
    f.write(f"Failed SMILES ({len(fail_single)}):\n")
    for smi in fail_single:
        f.write(f"  - {smi}\n")
    f.write(f"\nError SMILES ({len(err_single)}):\n")
    for smi in err_single:
        f.write(f"  - {smi}\n")
print(f"✓ Saved scalar results to: {results_path}")

print(f"\n📁 All files saved to: {output_dir.absolute()}")


✓ Saved per-conformer dataframe to: test_posebusters_output/per_conformer_df.csv
✓ Saved per-SMILES dataframe to: test_posebusters_output/per_smiles_df.csv
✓ Saved summary dataframe to: test_posebusters_output/summary_df.csv
✓ Saved scalar results to: test_posebusters_output/scalar_results.txt

📁 All files saved to: /auto/home/aram.dovlatyan/3DMolGen-new/3DMolGen/tests/evaluation/posebusters_check_tests/test_posebusters_output


## Detailed Analysis

Break down the results to understand what's happening.


In [28]:
# Detailed breakdown
print("=" * 80)
print("DETAILED ANALYSIS")
print("=" * 80)

print("\n📊 Per-SMILES Breakdown:")
for _, row in per_smiles_single.iterrows():
    print(f"\n  SMILES: {row['smiles']}")
    print(f"    - Number of conformers: {row['num_of_conformers']}")
    print(f"    - Pass percentage: {row['pass_percentage']:.2f}%")
    print(f"    - Error: {row.get('error', 'None')}")
    # Show boolean check columns
    bool_cols = [col for col in POSEBUSTERS_BOOL_COLUMNS if col in row.index]
    if bool_cols:
        print(f"    - Boolean checks:")
        for col in bool_cols:
            print(f"        {col}: {row[col]}")

print("\n\n📈 Summary Statistics:")
print(f"  - Total SMILES: {summary_single.iloc[0]['num_smiles']}")
print(f"  - Total conformers: {summary_single.iloc[0]['num_conformers']}")
print(f"  - Overall pass percentage: {summary_single.iloc[0]['pass_percentage']:.2f}%")

# Verify the math
weighted_total = float(
    (per_smiles_single["pass_percentage"] * per_smiles_single["num_of_conformers"]).sum()
)
total_confs = int(per_smiles_single["num_of_conformers"].sum())
calculated_overall = (weighted_total / total_confs) / 100.0

print(f"\n🔢 Math Verification:")
print(f"  - Weighted total: {weighted_total}")
print(f"  - Total conformers: {total_confs}")
print(f"  - Calculated overall: {calculated_overall:.6f}")
print(f"  - Returned overall: {overall_single:.6f}")
print(f"  - Match: {np.isclose(calculated_overall, overall_single, rtol=1e-6)}")


DETAILED ANALYSIS

📊 Per-SMILES Breakdown:

  SMILES: CC(C)(C)n1cn[nH]c1=S
    - Number of conformers: 3
    - Pass percentage: 33.33%
    - Error: posebusters_error
    - Boolean checks:
        mol_pred_loaded: 0.6666666666666666
        sanitization: 0.3333333333333333
        inchi_convertible: 0.6666666666666666
        all_atoms_connected: 0.3333333333333333

  SMILES: Cc1c(C#N)c2ccccc2n1C
    - Number of conformers: 1
    - Pass percentage: 0.00%
    - Error: 
    - Boolean checks:
        mol_pred_loaded: 1.0
        sanitization: 0.0
        inchi_convertible: 0.0
        all_atoms_connected: 0.0

  SMILES: NC(=O)C1CCC1
    - Number of conformers: 2
    - Pass percentage: 100.00%
    - Error: 
    - Boolean checks:
        mol_pred_loaded: 1.0
        sanitization: 1.0
        inchi_convertible: 1.0
        all_atoms_connected: 1.0


📈 Summary Statistics:
  - Total SMILES: 3.0
  - Total conformers: 6.0
  - Overall pass percentage: 50.00%

🔢 Math Verification:
  - Weighted tota